# Quicktour

PEFTは，大きな事前学習済みモデルのファインチューニングをする，パラメタ効率の良い方法を提供します．
従来のパラダイムでは，各ダウンストリームタスクのためにモデルの全てのパラメタをファインチューニングしていましたが，非常にコストがかかり，今日のモデルは莫大な数のパラメタを持つため現実的な方法ではありません．
そのかわり，より少量のプロンプトパラメタを学習したり，low-rank adaptation（LoRA）のような追加パラメタを利用することで，学習するパラメタの量を減らすことをします．

このクイックツアーでは，PEFTの主な機能や典型的なコンシューマデバイスではアクセスできないような大きなモデルでの学習や推論の方法を紹介します．

## 学習（Train）

各PEFT手法は，`PeftModel`を構築するための全ての重要なパラメタが保存されている`PeftConfig`クラスによって定義されます．
例えば，LoRAで学習するには，`LoraConfig`クラスを読み込み作成し，次に示すパラメタを指定します:

- `task_type`: 学習するタスク（ここではsequence-to-sequence言語モデルを扱います）
- `inference_mode`: 推論で使うかどうか
- `r`: 低次元行列（low-rank matrices）の次元
- `lora_alpha`: 低次元行列のスケーリング係数
- `lora_dropout`: LoRA層のドロップアウト確率

In [6]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

> [LoraConfig]()リファレンスで，他に調整可能なパラメタ（例えば，対象とするモジュール，バイアスのタイプなど）を確認してください．

`LoraConfig`を設定したら`PeftModel`を`get_peft_model()`で作成します．
この実行には，Transformersライブラリで読み込めるベースモデルと，LoRAで学習するためにモデルをどのように設定するかといったパラメタを含む`LoraConfig`が必要です．

ファインチューンするためのモデルを読み込みます:


In [7]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large")

ベースモデルと`peft_config`を`get_peft_config()`関数を使ってラップし，`PeftModel`を作成します．
学習可能なパラメタの数の感覚を得るために，`print_trainable_params()`を使います．

In [8]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


ベースの`bigscience/mt0-large`のパラメタ数1.2Bに対して，0.19%のパラメタのみを学習対象としています！

これで，Transformersの`Trainer`，`Accelerate`やカスタムPyTorch学習ループなどを使ってモデルを学習することができます．

例えば，`Trainer`クラスを使って学習する場合，`TrainingArguments`クラスをいくつかの学習ハイパーパラメータで設定します:

In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./your-name/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

モデル，学習パラメタ，データセット，トークナイザなど，必要なものを`Trainer`に私，`train()`を呼び出し学習を行います．

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=dataset["train"], 
    # eval_dataset=dataset["validation"],
    # tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

trainer.train()

/Users/fujie/anaconda3/envs/py311/lib/python3.11/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


ValueError: Trainer: training requires a train_dataset.

### モデルの保存

学習が終わったら，`save_pretrained()`関数を使って適当なディレクトリにモデルを保存します．

In [11]:
model.save_pretrained("output_dir")

`push_to_hub()`関数でHubに保存することもできます（先にHugging Faceアカウントでログインする必要があります）．

In [12]:
from huggingface_hub import notebook_login
notebook_login()
# model.push_to_hub("your-name/bigscience/mt0-large-lora")

何の方法も，学習された追加のPEFTの重みのみを保存します．つまり，保存，転送，読み込みを効率よく行うことができます．
例えば，LoRAで学習された`facebook/opt-350m`モデルは，`adapter_config.json`と`adapter_model.safetensors`の二つのファイルしか持ちません．
`adapter_model.safetensors`のファイルサイズはたったの6.3MBしかありません．
ベースモデルのパラメタは700MB程度のサイズです．

## 推論（Inference）

> `AutoPeftModel`クラスの利用可能な全リストを確認するには[AutoPeftModel](https://huggingface.co/docs/peft/package_reference/auto_class)APIリファレンスを参照してください．

PEFTで学習されたモデルを推論に使うためには，`AutoPeftModel`クラスの`from_pretrained()`関数を使ってモデルを読み込みます:
```python
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

model = AutoPeftModelForCausalLM.from_pretrained("ybelkada/opt-350m-lora")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

model = model.to("cuda")
model.eval()
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_length=50)
print(tokenizer.decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])
```

`AutoPeftModelFor`クラスで明示的にサポートされていないその他のタスク，例えば音声認識など，の場合，ベースの`AutoPeftModel`クラスを使ってモデルを読み込むことができます．
```python
from peft import AutoPeftModel

model = AutoPeftModel.from_pretrained("smangrul/openai-whisper-large-v2-LORA-colab")
```

## 次のステップ

PEFTを用いた一つの学習方法について学びました．
プロンプトチューニングなどのその他の方法を試してみることをお勧めします．
手順はこのクイックツアーで見たものと非常に似ています:

1. PEFTメソッドのために`PeftConfig`を設定します．
2. 設定とベースモデルから，`get_peft_model()`を使って`PeftModel`を作成します．

あとは好きなように学習をするだけです！
PEFTモデルを推論のために読み込むのには，`AutoPeftModel`クラスを使えます．

意味セグメンテーション（semantic segmentation），多言語音声認識（multilingual speech recognition），DreamBooth，トークン識別などなど，特定のタスクのためのPEFTメソッドでモデルを学習することに興味があれば，タスクガイドを見てください．